In [2]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
# magic command to display matplotlib plots inline within the ipython notebook
%matplotlib inline
# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import requests
import time
import catboost as cb

In [7]:
X = pd.read_csv('X_without_nulls.csv').drop(columns={'Unnamed: 0'},axis=1)

In [8]:
# отобранные тэги 
tags_list = ['shop','public_transport','office','cafe',
            'tourism','craft','amenity','building','sport','railway=station','education']

In [9]:
# базовые фичи
POINT_FEATURE_FILTERS = [
    ('tagged', lambda node: len(node.tags) > 0),
    ('railway', lambda node: node.tags.get('railway') == 'station'),
    ('shop', lambda node: 'shop' in node.tags),
    ('public_transport', lambda node: 'public_transport' in node.tags),
    ('office', lambda node: 'office' in node.tags),
    ('cafe', lambda node: 'cafe' in node.tags),
    ('tourism', lambda node: 'tourism' in node.tags),
    ('education', lambda node: 'education' in node.tags),    
]

# доп фичи
for i in tags_list:
    POINT_FEATURE_FILTERS.append((i, lambda node: i in node.tags))

In [10]:
POINT_FEATURE_FILTERS

[('tagged', <function __main__.<lambda>>),
 ('railway', <function __main__.<lambda>>),
 ('shop', <function __main__.<lambda>>),
 ('public_transport', <function __main__.<lambda>>),
 ('office', <function __main__.<lambda>>),
 ('cafe', <function __main__.<lambda>>),
 ('tourism', <function __main__.<lambda>>),
 ('education', <function __main__.<lambda>>),
 ('shop', <function __main__.<lambda>>),
 ('public_transport', <function __main__.<lambda>>),
 ('office', <function __main__.<lambda>>),
 ('cafe', <function __main__.<lambda>>),
 ('tourism', <function __main__.<lambda>>),
 ('craft', <function __main__.<lambda>>),
 ('amenity', <function __main__.<lambda>>),
 ('building', <function __main__.<lambda>>),
 ('sport', <function __main__.<lambda>>),
 ('railway=station', <function __main__.<lambda>>),
 ('education', <function __main__.<lambda>>)]

In [11]:
X.head()

,address,address_rus,atm_group,id,isTrain,lat,long,target
0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",32.0,8526.0,True,46.940995,142.738319,0.011500
1,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",32.0,8532.0,True,46.937353,142.753348,0.029710
2,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",32.0,8533.0,True,46.959413,142.741113,0.009540
3,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",32.0,8684.0,True,55.805827,37.515146,-0.094035
4,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",32.0,37.0,True,69.343541,88.211228,0.079277


In [13]:
# центры квадратов в виде матрицы
X_zone_centers = X[['lat', 'long']].as_matrix()

In [15]:
for prefix, point_filter in POINT_FEATURE_FILTERS:
    
    print(prefix)
    
    # берем подмножество точек в соответствии с фильтром
    coords = np.array([
        [node.lat, node.lon]
        for node in tagged_nodes
        if point_filter(node)
    ])

    # строим структуру данных для быстрого поиска точек
    neighbors = NearestNeighbors().fit(coords)
    
    # признак вида "количество точек в радиусе R от центра квадрата"
    for radius in [0.001, 0.003, 0.005, 0.007, 0.01]:
        dists, inds = neighbors.radius_neighbors(X=X_zone_centers, radius=radius)
        df_features['{}_points_in_{}'.format(prefix, radius)] = np.array([len(x) for x in inds])

    # признак вида "расстояние до ближайших K точек"
    for n_neighbors in [3, 5, 10, 15, 21]:
        dists, inds = neighbors.kneighbors(X=X_zone_centers, n_neighbors=n_neighbors)
        df_features['{}_mean_dist_k_{}'.format(prefix, n_neighbors)] = dists.mean(axis=1)
        df_features['{}_max_dist_k_{}'.format(prefix, n_neighbors)] = dists.max(axis=1)
        df_features['{}_std_dist_k_{}'.format(prefix, n_neighbors)] = dists.std(axis=1)

    # признак вида "расстояние до ближайшей точки"
    df_features['{}_min'.format(prefix)] = dists.min(axis=1)

tagged


NameError: name 'tagged_nodes' is not defined

In [322]:
## OSM
import requests
import json

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="RU"][admin_lev el=2];
(node["amenity"="cafe"](area);
 way["amenity"="cafe"](area);
 rel["amenity"="cafe"](area);
);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
#data = response.json()

## список отделений Росбанка

In [138]:
from bs4 import BeautifulSoup

In [139]:
import requests
session = requests.Session() 
url = 'https://bankiros.ru/bank/rosbank/departments-tab' 

In [ ]:
import urllib2
from bs4 import BeautifulSoup
page = urllib2.urlopen('some_site').read()
soup = BeautifulSoup(page)
body = soup.find('body')
the_contents_of_body_without_body_tags = body.findChildren()

In [ ]:
soup = BeautifulSoup(page)

In [140]:
request = session.get(url)

In [141]:
text = request.text

In [142]:
soup = BeautifulSoup(text)

In [143]:
body = soup.find('body').find('div', {'class': 'container main-container'}).find('section').find('div', {'class': 'wrapper'})

In [144]:
body2 = body.find('div', {'class': 'mapobject_table clearfix mr-m-20'})

In [145]:
body3 = body2.find('div', {'id': 'pjg_w0'}).find('div', {'id': 'w0'})

In [146]:
body4 = body3.find('table', {'class': 'standard products dep-tab'})

In [147]:
def cond(x):
    if x:
        return x.startswith("dep_adress col-sm-6")
    else:
        return False

In [148]:
tr = body4.find('tbody')
td = tr.findAll('td', {'class': cond})

In [149]:
import re

In [152]:
for i in range(len(td)):
    print str(td[i]).split('>')[1][:-4]

Московская область, Дубна, проспект Боголюбова, 31
Республика Башкортостан, Салават, Октябрьская улица, 31
Архангельская область, Новодвинск, улица Ворошилова, 3
Алтайский край, Славгород, улица Володарского, 143/1
Приморский край, Партизанск, Ленинская улица, 1
Новгородская область, Боровичи, улица Подбельского, 36
Омская область, Называевск, улица Ленина, 68
Хабаровский край, Вяземский, улица Козюкова, 5
Костромская область, Шарья, Водопроводный переулок, 5
Удмуртская Республика, Сарапул, Советская улица, 17А
Республика Калмыкия, Лагань, улица Баташова, 1
Амурская область, Шимановск, улица Крупской, 9
Владимирская область, Киржач, Советская улица
Курская область, Курчатов, Коммунистический проспект, 7
143968, Московская область, Реутов, улица Победы, 30
Приморский край, Спасск-Дальний, Советская улица, 41
Оренбургская область, Орск, проспект Ленина, 93
Орловская область, Ливны, улица Карла Маркса, 139
Амурская область, Сковородино, улица Победы, 17
Московская область, Серпухов, 1-я М

In [159]:
import pandas as pd
departments = pd.DataFrame(columns={'Address'})

In [170]:
j=0
for page in range(1,22):
    url = 'https://bankiros.ru/bank/rosbank/departments-tab?page=%s' %(page)
    print url
    request = session.get(url)
    text = request.text
    soup = BeautifulSoup(text)
    body = soup.find('body').find('div', {'class': 'container main-container'}).find('section').find('div', {'class': 'wrapper'})
    body2 = body.find('div', {'class': 'mapobject_table clearfix mr-m-20'})
    body3 = body2.find('div', {'id': 'pjg_w0'}).find('div', {'id': 'w0'})
    body4 = body3.find('table', {'class': 'standard products dep-tab'})
    tr = body4.find('tbody')
    td = tr.findAll('td', {'class': cond})
    for i in range(len(td)):
        address = str(td[i]).split('>')[1][:-4]
        departments.loc[j] = address
        j+=1

https://bankiros.ru/bank/rosbank/departments-tab?page=1
https://bankiros.ru/bank/rosbank/departments-tab?page=2
https://bankiros.ru/bank/rosbank/departments-tab?page=3
https://bankiros.ru/bank/rosbank/departments-tab?page=4
https://bankiros.ru/bank/rosbank/departments-tab?page=5
https://bankiros.ru/bank/rosbank/departments-tab?page=6
https://bankiros.ru/bank/rosbank/departments-tab?page=7
https://bankiros.ru/bank/rosbank/departments-tab?page=8
https://bankiros.ru/bank/rosbank/departments-tab?page=9
https://bankiros.ru/bank/rosbank/departments-tab?page=10
https://bankiros.ru/bank/rosbank/departments-tab?page=11
https://bankiros.ru/bank/rosbank/departments-tab?page=12
https://bankiros.ru/bank/rosbank/departments-tab?page=13
https://bankiros.ru/bank/rosbank/departments-tab?page=14
https://bankiros.ru/bank/rosbank/departments-tab?page=15
https://bankiros.ru/bank/rosbank/departments-tab?page=16
https://bankiros.ru/bank/rosbank/departments-tab?page=17
https://bankiros.ru/bank/rosbank/departm

In [172]:
departments.to_csv('rosbank_departments.csv')

## список отделений Сбербанка

In [306]:
sber = pd.read_excel('branches-2.xls')

In [307]:
sber = sber.drop(columns = sber.columns[0:8]).drop(columns = sber.columns[12:])

In [308]:
sber = sber.reset_index().drop(columns = {'index'})
sber = sber.drop(sber.index[:3]).reset_index().drop(columns = {'index'})

In [309]:
sber = sber.dropna(axis=0, how='any')

In [310]:
sber.head()

,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Белгородская область,г.Белгород,улица Князя Трубецкого,24
1,Белгородская область,г.Белгород,улица Некрасова,"5/8, корп.Б"
2,Белгородская область,г.Алексеевка,переулок 1-й К. Маркса,2
10,Белгородская область,г.Алексеевка,улица Комсомольская,46А
13,Белгородская область,с.Жуково,улица Центральная,66


In [311]:
sber['Unnamed: 9'] = sber['Unnamed: 9'].apply(lambda x: x.encode('utf-8'))
sber['Unnamed: 10'] = sber['Unnamed: 10'].apply(lambda x: x.encode('utf-8'))
sber['Unnamed: 11'] = sber['Unnamed: 11'].apply(lambda x: x.encode('utf-8'))

In [312]:
sber.head()

,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Белгородская область,г.Белгород,улица Князя Трубецкого,24
1,Белгородская область,г.Белгород,улица Некрасова,"5/8, корп.Б"
2,Белгородская область,г.Алексеевка,переулок 1-й К. Маркса,2
10,Белгородская область,г.Алексеевка,улица Комсомольская,46А
13,Белгородская область,с.Жуково,улица Центральная,66


In [286]:
#sber['Unnamed: 9'][13].startswith('с')

In [287]:
#sber['Unnamed: 9'].apply(lambda x: x.startswith('с',''))

In [288]:
a.startswith('c')

False

In [313]:
def delete_c(address):
    if address.startswith('г'):
        return address
    else:
        return 'село'
    
sber['Unnamed: 9']  = sber['Unnamed: 9'].apply(lambda x: delete_c(x))

In [314]:
sber = sber[sber['Unnamed: 9'] != 'село']

In [319]:
sber = sber.drop_duplicates()
sber.head(10)

,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,address
0,Белгородская область,г.Белгород,улица Князя Трубецкого,24,г.Белгород улица Князя Трубецкого 24
1,Белгородская область,г.Белгород,улица Некрасова,"5/8, корп.Б","г.Белгород улица Некрасова 5/8, корп.Б"
2,Белгородская область,г.Алексеевка,переулок 1-й К. Маркса,2,г.Алексеевка переулок 1-й К. Маркса 2
10,Белгородская область,г.Алексеевка,улица Комсомольская,46А,г.Алексеевка улица Комсомольская 46А
17,Белгородская область,г.Белгород,улица Железнякова,18-А,г.Белгород улица Железнякова 18-А
24,Белгородская область,г.Бирюч,улица Карла Маркса,4,г.Бирюч улица Карла Маркса 4
35,Белгородская область,г.Белгород,проспект Б.Хмельницкого,160,г.Белгород проспект Б.Хмельницкого 160
37,Белгородская область,г.Алексеевка,улица Ст. Разина,56,г.Алексеевка улица Ст. Разина 56
39,Белгородская область,г.Белгород,улица Генерала Апанасенко,97,г.Белгород улица Генерала Апанасенко 97
40,Белгородская область,г.Белгород,улица Буденого,2-А,г.Белгород улица Буденого 2-А


In [316]:
sber['address'] = sber['Unnamed: 9'] + ' ' + sber['Unnamed: 10'] + ' ' + sber['Unnamed: 11']

In [321]:
pd.DataFrame(sber['address']).to_csv('sber_departments.csv')